In [1]:
import torch
import torch.nn as nn

In [7]:
class FNN_plus(nn.Module):
    def __init__(self,n_features,dim=128):
        super(FNN_plus,self).__init__()
        self.features = nn.Embedding(n_features,dim,max_norm=1)
        self.mlp_layer = self.__mlp(dim)

    def __mlp(self,dim):
        return nn.Sequential(
            nn.Linear(dim,dim//2),
            nn.Tanh(),
            nn.Linear(dim//2,dim//4),
            nn.Tanh(),
            nn.Linear(dim//4,1),
            nn.Sigmoid()
        )

    def FMaggregator(self,feature_embs):
        square_of_sum = torch.sum(feature_embs,dim=1)**2
        sum_of_square = torch.sum(feature_embs**2,dim=1)
        output = square_of_sum - sum_of_square
        return output

    def __getAllFeatures(self,u,i,user_df,item_df):
        users = torch.LongTensor(user_df.loc[u].values)
        items = torch.LongTensor(item_df.loc[i].values)
        total = torch.cat([users,items],dim=1)
        return total

    def forward(self,u,i,user_df,item_df):
        all_feature_index = self.__getAllFeatures(u,i,user_df,item_df)
        all_feature_embs = self.features(all_feature_index)
        out = self.FMaggregator(all_feature_embs)
        out = self.mlp_layer(out)
        out = torch.squeeze()
        return out